In [ ]:
# default_exp utils

# utils

> Assorted low-level utilities for a flexible SEC filings scanner.

In [ ]:
#hide
%load_ext autoreload
%autoreload 2
from nbdev import show_doc

In [ ]:
#export

from bs4 import BeautifulSoup
import datetime
import gzip
import os
import pickle
import re
from pytz import timezone
import requests
import time
import webbrowser

## Set directory for scraped data
We store all scraped data under stockDataRoot.

In [ ]:
#export
stockDataRoot = os.path.expanduser(os.path.join('~','secData2'))
def setStockDataRoot(loc) :
    "Set location for storing scraped stock data."
    global stockDataRoot
    stockDataRoot = loc

In [ ]:
print(stockDataRoot)

C:\Users\ikedi\secData2


## Low-level download functions
We download SEC data using the requests library.
We retry a few times in case of a temporary internet glitch,
and also recognize an SEC-specific temporary outage message and raise an Exception for it
so that we can flag the problem and retry later.

In [ ]:
#export

def requestUrl(url, timeout=5.0, nTries=5, returnText=False, **kwargs) :
    "Downloads a URL using the requests package."
    for i in range(nTries) :
        try :
            r = requests.get(url,timeout=timeout,params=kwargs)
            r.raise_for_status()
            return r.text if returnText else r
        except Exception as e :
            print('Error','downloading',url,'-',e,'; retrying ...')
            if i >= nTries-1 :
                raise

accessNoPatStr = r'\d{10}-\d+-\d+'
accessNoPat = re.compile(accessNoPatStr)
def secIndexUrl(accessNo, includePref=False) :
    "Returns the url for the index page of an SEC filing specified by accession number."
    return ((secUrlPref if includePref else '')
            + '/Archives/edgar/data/'+accessNo.replace('-','')
            +'/'+accessNo+'-index.htm')

secUrlPref = 'https://www.sec.gov'
pageUnavailablePat = re.compile('page is temporarily unavailable',re.IGNORECASE)
def downloadSecUrl(secSubUrlOrAccessNo, soupify=False) :
    """
    Downloads a page from the SEC site. The page can be specified by
    a sub-URL (ex. /cgi-bin/browse-edgar?CIK=0000716314&owner=exclude),
    or just by an accession number (ex. 0001193125-21-181366), in which
    case the index page for that filing is downloaded.
    
    Optionally parses the page contents into a BeautifulSoup object.

    Checks for an SEC-specific temporary outage message, and raises an
    Exception if it's detected, so that we can possibly flag to retry
    the download later.
    """
    if accessNoPat.match(secSubUrlOrAccessNo) :
        secSubUrl = secIndexUrl(secSubUrlOrAccessNo)
    else :
        secSubUrl = secSubUrlOrAccessNo
    urlContents = requestUrl(secUrlPref+secSubUrl).text
    if pageUnavailablePat.search(urlContents) :
        raise Exception('temporary SEC outage')
    if soupify :
        return BeautifulSoup(urlContents,'html.parser')
    return urlContents

spacesPat = re.compile(r'\s+')
def getCombSoupText(tag) :
    "Get the combined text from a BeautifulSoup tag."
    return spacesPat.sub(" "," ".join(tag.stripped_strings))

In [ ]:
accessNoPat.match('000119315-21-181366')

Test downloading from the SEC website:

In [ ]:
t = downloadSecUrl('')
assert 'securities and exchange' in t.lower()

In [ ]:
testUrl = secIndexUrl('0000909108-21-000051')
print('URL:',testUrl)
s = downloadSecUrl(testUrl, soupify=True)
s = s.find('span','companyName')
print('Company name HTML:',s)
companyName = getCombSoupText(s)
print('Company name text:',companyName)
assert companyName.lower().startswith('diamond hill')

URL: /Archives/edgar/data/000090910821000051/0000909108-21-000051-index.htm
Error downloading https://www.sec.gov/Archives/edgar/data/000090910821000051/0000909108-21-000051-index.htm - 403 Client Error: Forbidden for url: https://www.sec.gov/Archives/edgar/data/000090910821000051/0000909108-21-000051-index.htm ; retrying ...
Company name HTML: <span class="companyName">DIAMOND HILL INVESTMENT GROUP INC (Filer)
 <acronym title="Central Index Key">CIK</acronym>: <a href="/cgi-bin/browse-edgar?CIK=0000909108&amp;action=getcompany">0000909108 (see all company filings)</a></span>
Company name text: DIAMOND HILL INVESTMENT GROUP INC (Filer) CIK : 0000909108 (see all company filings)


## Low-level functions for storing scraped data
We store scraped data in pickled format,
either storing an object in a single pickled file
or storing a dict by saving one file per key
(for example, one file per date).
We can optionally use gzip compression (smaller files, but slower to read).

In [ ]:
#export

def openFp(fpath, mode, use_gzip) :
    "Open a file for writing or reading, optionally using gzip compression."
    openfunc = gzip.open if use_gzip else open
    return openfunc(fpath,mode)

def pickSave(fpath, ob, use_gzip=False, **kwargs) :
    "Save a pickled object to a file, optionally using gzip compression."
    with openFp(fpath, 'wb', use_gzip) as f :
        pickle.dump(ob, f, **kwargs)

def pickLoad(fpath, use_gzip=False) :
    "Load a pickled object from a file, optionally using gzip compression."
    with openFp(fpath, 'rb', use_gzip) as f :
        return pickle.load(f)

def pickLoadIfPath(path_or_ob, use_gzip=False) :
    """
    If given a path, loads a pickled object from it; otherwise returns
    its argument unchanged (assumes it's an already loaded object).
    """
    if isinstance(path_or_ob,str) :
        return pickLoad(path_or_ob, use_gzip=use_gzip)
    else :
        return path_or_ob

Test pickled data storage in single files:

In [ ]:
import random
rng = random.Random(42)
test_rand = dict((f'r{i}', rng.random()) for i in range(10))
pickSave('test.pkl', test_rand)
assert test_rand == pickLoad('test.pkl')
pickSave('test.pkl', test_rand, use_gzip=True)
assert test_rand == pickLoad('test.pkl', use_gzip=True)
time.sleep(1)
os.unlink('test.pkl')

In [ ]:
#export

def savePklToDir(toDir, fName, ob, use_gzip=False, **kwargs) :
    """
    Saves a pickled object to a file under a directory, optionally using gzip compression.
    Creates the directory if it doesn't exist.
    """
    if not os.path.exists(toDir) :
        os.makedirs(toDir)
    fPath = os.path.join(toDir, fName)
    pickSave(fPath, ob, use_gzip=use_gzip, **kwargs)

def loadPklFromDir(fromDir, fName, defaultVal, use_gzip=False) :
    """
    Load a pickled object from a file under a directory, optionally using gzip compression.
    Returns a default value if the file doesn't exist.
    """
    fPath = os.path.join(fromDir, fName)
    if os.path.exists(fPath) :
        return pickLoad(fPath, use_gzip=use_gzip)
    else :
        return defaultVal

Test pickled data storage under directory:

In [ ]:
savePklToDir('testdirpkl','test.pkl', test_rand)
assert test_rand == loadPklFromDir('testdirpkl','test.pkl',None)
time.sleep(1)
os.unlink(os.path.join('testdirpkl','test.pkl'))
os.rmdir('testdirpkl')

In [ ]:
#export

def saveSplitPklToDir(m, toDir, fSuff='m.pkl', dirtyMap=None, use_gzip=False, **kwargs) :
    """
    Saves a dict with str keys to a separate file for each key.
    If dirtyMap is True, saves all keys.
    If dirtyMap is None (default), saves only keys that don't yet have a file saved.
    Otherwise, also saves keys k for which dirtyMap.get(k) is true.
    """
    if not os.path.exists(toDir) :
        os.makedirs(toDir)
    for k in sorted(m.keys()) :
        fPath = os.path.join(toDir, k+fSuff)
        if dirtyMap is True :
            needToSave = True
        else :
            needToSave = not os.path.exists(fPath)
            if dirtyMap is not None :
                needToSave = needTooSave or dirtyMap.get(k)
        if needToSave :
            pickSave(fPath, m[k], use_gzip=use_gzip, **kwargs)

def loadSplitPklFromDir(fromDir, startK=None, endK=None, fSuff='m.pkl') :
    """
    Loads a pickled dict with str keys stored with a separate file for each key,
    optionally restricting to keys in [startK .. endK)
    """
    m = {}
    if not os.path.exists(fromDir) :
        return m
    fNames = sorted(fName for fName in os.listdir(fromDir)
                    if fName.endswith(fSuff))
    for fName in fNames :
        fPref = fName[:-len(fSuff)]
        if ((startK is not None and fPref<startK)
                or (endK is not None and endK<=fPref)) :
            continue
        m[fPref] = pickLoad(os.path.join(fromDir,fName))
    return m

Test pickled dict storage split by key:

In [ ]:
saveSplitPklToDir(test_rand, 'testsplitpkl')
assert test_rand == loadSplitPklFromDir('testsplitpkl')
test_sub = dict((k,v) for k,v in test_rand.items() if 'r3'<=k<'r7')
assert test_sub == loadSplitPklFromDir('testsplitpkl',startK='r3',endK='r7')
time.sleep(1)
for k in test_rand.keys() :
    os.unlink(os.path.join('testsplitpkl',k+'m.pkl'))
os.rmdir('testsplitpkl')

## YYYYMMDD Date strings
A few functions for working date string in the format YYYYMMDD, as used in some SEC URLs.

In [ ]:
#export

def toDateStr(d=None) :
    """
    Converts date object or ISO format date string to YYYYMMDD format string;
    leaves YYYYMMDD format strings unchanged;
    None -> today.
    """
    if isinstance(d,str) :
        dateStr = d
    else :
        if d is None :
            d = curEasternUSTime()
        dateStr = d.isoformat()[:10]
    return dateStr.replace('-','').replace('/','')

dateStr8Pat = re.compile(r"(\d\d\d\d)(\d\d)(\d\d)$")
def toDate(d=None) :
    """
    Converts date string in ISO or YYYYMMDD format to date object;
    leaves date objects unchanged;
    None -> today.
    """
    if isinstance(d,str) :
        dateStr = d.replace('-','').replace('/','')
        m = dateStr8Pat.match(dateStr)
        if m is None :
            raise Exception('invalid date str "'+d+'"')
        return datetime.date(int(m.group(1)),int(m.group(2)),int(m.group(3)))
    if d is None :
        return curEasternUSTime()
    return d

def isWeekend(d) :
    "Says if date string or date object is on a weekend (Saturday or Sunday)."
    return toDate(d).weekday() >= 5

def dateStrsBetween(d1,d2=None,excludeWeekends=False) :
    """
    Returns a list of date strings in YYYYMMDD format from d1 (inclusive)
    to d2 (exclusive), optionally excluding weekends.
    """
    d1 = toDate(d1)
    d2Str = toDateStr(d2)
    res = []
    while True :
        d1Str = toDateStr(d1)
        if d1Str >= d2Str :
            break
        if not (excludeWeekends and isWeekend(d1)) :
            res.append(d1Str)
        d1 = d1 + datetime.timedelta(1)
    return res

def formatDateStr(dStr,sep='-') :
    "Convert YYYYMMDD format date string to YYYY-MM-DD."
    return sep.join((dStr[:4],dStr[4:6],dStr[6:8]))

Test YYYYMMDD date string functions:

In [ ]:
assert dateStrsBetween('20201230','20210103')==['20201230', '20201231', '20210101', '20210102']
assert dateStrsBetween('20201231','20210106',excludeWeekends=True)==['20201231', '20210101', '20210104', '20210105']
assert formatDateStr('20200630')=='2020-06-30'
assert formatDateStr('20200630','/')=='2020/06/30'
assert (isWeekend('20210605'),isWeekend('20210606'),isWeekend('20210607')) == (True,True,False)

## Get current Eastern US time
This is used to control when to check for SEC filings.

In [ ]:
#export

easternUSTimeZone = timezone('US/Eastern')
def curEasternUSTime() :
    return datetime.datetime.now(easternUSTimeZone)

In [ ]:
print(curEasternUSTime().isoformat())

2021-06-05T12:41:11.647654-04:00


## Some functions for debugging

In [ ]:
#export

def secBrowse(accessNo) :
    "Open the index page of an SEC filing specified by accession number in a web browser."
    webbrowser.open_new_tab(secIndexUrl(accessNo,True))

def printSamp(m,n=10) :
    """
    Prints a sample of n items from object m , where m is a list or dict;
    for other objects just prints the whole thing.
    """
    if isinstance(m,list) :
        for i,item in enumerate(m[:n]) :
            print(i,end=' ')
            printSamp(item,n)
    elif isinstance(m,dict) :
        for k in m.keys()[:n] :
            print(k,end=' ')
            printSamp(m[k],n)
    else :
        print(m)

def printErrInfoOrAccessNo(msg,infoOrAccessNo) :
    print(msg,end=' ')
    if isinstance(infoOrAccessNo,str) and accessNoPat.match(infoOrAccessNo) :
        print(secIndexUrl(infoOrAccessNo,True))
    else :
        print(repr(infOrAccessNo))

In [ ]:
#hide
# uncomment and run to regenerate all library Python files
# from nbdev.export import notebook2script; notebook2script()